In [1]:
import matplotlib.pyplot as plt

from numpy.linalg import eig
from scipy.linalg import expm, eig

from primitives.primitives import *

In [2]:
def commute(a, b):
    a = np.array(a, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128)

    c = a@b
    d = b@a
    if np.allclose(c-d, np.zeros(c.shape)):
        print("Commute")
    elif np.allclose(c+d, np.zeros(c.shape)):
        print("Anti-commute")
    else:
        print("Null")
        
def is_unitary(m):
    return np.allclose(np.eye(m.shape[0]), m.dot(m.conj().T))

def is_hermitian(m):
    m = np.array(m, dtype=np.complex128)
    return np.allclose(m, m.conj().T)

def is_diagonal(m):
    m = np.array(m, dtype=np.complex128)
    row, cols = m.shape
    for r in range(row):
        for c in range(cols):
            if r != c:
                if m[r][c] != 0:
                    return False
    return True

In [3]:
def find_unitary_transform(A, B):
    """
    Find the unitary matrix U such that U A U^dagger = B.

    Parameters:
    A (ndarray): Input matrix A.
    B (ndarray): Input matrix B.

    Returns:
    U (ndarray or None): Unitary matrix U if it exists, otherwise None.
    """
    A = np.array(A, complex)
    B = np.array(B, complex)
    
    # Get the eigenvalues and eigenvectors of A
    eigvals_A, eigvecs_A = eig(A)
    eigvals_B, eigvecs_B = eig(B)
    
    # Sort the eigenvalues and corresponding eigenvectors for comparison
    idx_A = np.argsort(eigvals_A)
    idx_B = np.argsort(eigvals_B)
    
    eigvals_A_sorted = eigvals_A[idx_A]
    eigvals_B_sorted = eigvals_B[idx_B]
    
    eigvecs_A_sorted = eigvecs_A[:, idx_A]
    eigvecs_B_sorted = eigvecs_B[:, idx_B]

    # Check if the sorted eigenvalues match
    if not np.allclose(eigvals_A_sorted, eigvals_B_sorted):
        print("not all eigenvalues match")
        print(eigvals_A_sorted)
        print(eigvals_B_sorted)
        return None

    eigvecs_A_sorted = np.linalg.qr(eigvecs_A_sorted)[0]
    eigvecs_B_sorted = np.linalg.qr(eigvecs_B_sorted)[0]

    # Construct the unitary matrix U
    U = eigvecs_B_sorted @ eigvecs_A_sorted.conj().T
    
    # Check if U is unitary
    if np.allclose(U @ U.conj().T, np.eye(U.shape[0])):
        return U
    else:
        print("not unitary")
        return U

In [4]:
def to_latex(mat):
    text = r"\begin{pmatrix}"
    text += "\n"
    for row in mat:
        for idx, el in enumerate(row):
            imag = False
            
            if np.imag(el) != 0 and np.real(el) != 0:
                raise Exception
            if np.imag(el) != 0:
                imag = True
                el = np.imag(el)
            else:
                el = np.real(el)
            if el%1 == 0:
                text += str(int(el))
            elif np.abs(el) == 0.7071068:
                if el < 0:
                    text += "-"
                if imag:
                    text += "i"
                text += r"\sqrt{0.5}"
            else:
                text += str(el)
            
            if idx + 1 < len(row):
                text += " & "
        text += r"\\"
        text += "\n"
        

    text += r"\end{pmatrix}"
    return text

## "Unitary conjugation"

\begin{equation}
e^{A \otimes B} = U^\dagger e^{D\otimes B} U
\end{equation}

with D diagonal and A hermitian

In [5]:
A = 1j * (
    TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_2) -
    TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_1)
)

B = sy_gamma_5

print(f"A is hermitian: {is_hermitian(A)}")

D = (
    TensorProduct(sy_gamma_5, sy_id) +
    TensorProduct(sy_id, sy_gamma_5)
)
print(f"D is diagonal: {is_diagonal(D)}")

# find U
U = find_unitary_transform(A, D)

# check U does the correct transformation

e_U = np.array(TensorProduct(U, sy_id), dtype=np.complex128)

left = expm(np.array(TensorProduct(A, B), dtype=np.complex128))
right = expm(np.array(TensorProduct(D, B), dtype=np.complex128))
right = e_U.conj().T @ right @ e_U

print(f"Method worked :{np.allclose(left, right)}")

#print(to_latex(np.round(U, 7)))

A is hermitian: True
D is diagonal: True
Method worked :True


\begin{equation}
U=
\begin{pmatrix}
0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0\\
\end{pmatrix}
\end{equation}

In [8]:
A = (
    TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_5*sy_gamma_2) -
    TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_5*sy_gamma_1)
)

B = TensorProduct(sy_gamma_1, sy_gamma_2)

print(f"A is hermitian: {is_hermitian(A)}")

D = (
    TensorProduct(sy_gamma_5, sy_id) +
    TensorProduct(sy_id, sy_gamma_5)
)
print(f"D is diagonal: {is_diagonal(D)}")

# find U
U = find_unitary_transform(A, D)

# check U does the correct transformation

e_U = np.array(TensorProduct(U, sy_id, sy_id), dtype=np.complex128)

left = expm(np.array(TensorProduct(A, B), dtype=np.complex128))
right = expm(np.array(TensorProduct(D, B), dtype=np.complex128))
right = e_U.conj().T @ right @ e_U

print(f"Method worked :{np.allclose(left, right)}")

#print(to_latex(np.round(U, 7)))

A is hermitian: True
D is diagonal: True
Method worked :True


\begin{equation}
V = \begin{pmatrix}
0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
\end{pmatrix}
\end{equation}

In [9]:
A = TensorProduct(sy_gamma_1, sy_gamma_2)

B = (
    TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_5*sy_gamma_2) -
    TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_5*sy_gamma_1)
)

print(f"A is hermitian: {is_hermitian(A)}")

D = TensorProduct(sy_gamma_5, sy_id)

print(f"D is diagonal: {is_diagonal(D)}")

# find U
U = find_unitary_transform(A, D)

# check U does the correct transformation

e_U = np.array(TensorProduct(U, sy_id, sy_id), dtype=np.complex128)

left = expm(np.array(TensorProduct(A, B), dtype=np.complex128))
right = expm(np.array(TensorProduct(D, B), dtype=np.complex128))
right = e_U.conj().T @ right @ e_U

print(f"Method worked :{np.allclose(left, right)}")

#print(to_latex(np.round(U, 7)))

A is hermitian: True
D is diagonal: True
Method worked :True


\begin{equation}
W
\begin{pmatrix}
-i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0\\
0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0\\
0 & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0\\
\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0\\
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5}\\
0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5}\\
\end{pmatrix}
\end{equation}